# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

Load and/or check for needed environmental variables

In [11]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****4abe
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ad-6


In [12]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [13]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [14]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [15]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## Build a Dynamic Prompt
Utilize runtime context and middleware to generate a dynamic prompt.

In [16]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [17]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b",
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [18]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (2b21d9b7-ac45-4f52-b535-0a16da83f4bd)
 Call ID: 2b21d9b7-ac45-4f52-b535-0a16da83f4bd
  Args:
    query: SELECT t.Name, t.UnitPrice
FROM Track t
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Artist ar ON a.ArtistId = ar.ArtistId
WHERE ar.Name = 'Frank Harris'
ORDER BY t.UnitPrice DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql


================================== Ai Message ==================================

I’m sorry, but the tables that contain purchase information (such as Invoice, InvoiceLine, or Customer) aren’t available in the current schema, so I can’t determine who “Frank Harris” is or what he purchased.  

If you have a different table or dataset that includes purchase records, l

In [19]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (8e0f926a-29b3-4a66-8c94-56f8c57ea9f0)
 Call ID: 8e0f926a-29b3-4a66-8c94-56f8c57ea9f0
  Args:
    query: PRAGMA table_info(purchases);
================================= Tool Message =================================
Name: execute_sql


================================== Ai Message ==================================
Tool Calls:
  execute_sql (fbb31031-68ff-4837-8b36-6c90eceec6aa)
 Call ID: fbb31031-68ff-4837-8b36-6c90eceec6aa
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]